<a href="https://colab.research.google.com/github/Jake-Baum/model-gen/blob/main/PolyGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [44]:
import tensorflow as tf
import numpy as np
import urllib.request
import zipfile
import os

Load and unzip files

Pulling in a pre-decimated subset of the ShapeNetCore data set.

In [45]:
def unzip(readPath, writePath):
    with zipfile.ZipFile(readPath, 'r') as zip_ref:
        zip_ref.extractall(writePath)

url = 'https://masonmcgough-data-bucket.s3-us-west-2.amazonaws.com/ShapeNetCore_PolyGenSubset.zip'

urllib.request.urlretrieve(url, 'dataset.zip')
unzip('dataset.zip', 'dataset')

urllib.request.urlretrieve('https://masonmcgough-data-bucket.s3-us-west-2.amazonaws.com/cube.obj', 'cube.obj')

os.listdir('.')

['.config', 'cube.obj', 'dataset', 'dataset.zip', 'sample_data']

Create method for loading objects from file into array of vertices.

A simple cube model is loaded

In [23]:
def loadObject(filename):
  vertices = []
  with open(filename, 'r') as mesh:
    for line in mesh:
      data = line.split()
      if len(data) > 0 and data[0] == 'v':
        vertices.append(data[1:])
  return np.array(vertices, dtype=np.float32)

verts = loadObject('cube.obj')
#verts = loadObject('dataset/train/chair/model_012429-0001.obj')

Sort the vertices, in ascending order, z axis first, then y, then x

In [26]:
vertsKeys = [verts[..., i] for i in range(verts.shape[-1])]
sortIdxs = np.lexsort(vertsKeys)
vertsSorted = verts[sortIdxs]
flattenedVerts = [coord for vert in vertsSorted for coord in vert]
flattenedVerts

[-1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

Normalise (-1.0 - 1.0) and quantise (0, 255) vertices.

In [40]:
normVerts = [(i - min(flattenedVerts)) / (max(flattenedVerts) - min(flattenedVerts)) for i in flattenedVerts]

nBits = 8
nVals = 2 ** nBits
delta = 1.0 / nVals
quantVerts = np.maximum(np.minimum([i // delta for i in normVerts], nVals - 1), 0).astype(np.int32)
quantVerts

array([  0,   0,   0, 255,   0,   0,   0, 255,   0, 255, 255,   0,   0,
         0, 255, 255,   0, 255,   0, 255, 255, 255, 255, 255], dtype=int32)

Create tokens.  There are 3 sets of tokens:
 - The value tokens, which is the start token, followed by the quantised vertices (with the number of tokens added to the value, such that they do not overlap), then the end token, followed by any padding tokens.
 - The coordinate tokens, which label the value tokens with the coordinate index, 1 for x, 2 for y and 3 for z.
 - The position tokens, which is simply a number range of equal length to the previous lists.

In [38]:
TOKENS = {
    '<pad>': 0,
    '<sos>': 1,
    '<eos>': 2
}

maxVerts = 12
maxSeqLen = 3 * maxVerts + 2 # number of coords + start and stop tokens
nTokens = len(TOKENS)
seqLen = len(quantVerts) + 2
nPadding = maxSeqLen - seqLen

coordTokens = np.concatenate(([0], np.arange(len(quantVerts)) % 3 + 1, (nPadding + 1) * [0]))
print(coordTokens, len(coordTokens))

valTokens = np.concatenate((
    [TOKENS['<sos>']],
    quantVerts + nTokens,
    [TOKENS['<eos>']],
    nPadding * [TOKENS['<pad>']]
))
print(valTokens, len(valTokens))

posTokens = np.arange(len(coordTokens), dtype=np.int32)
posTokens

[0 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 0 0 0 0 0 0 0 0 0 0 0 0
 0] 38
[  1   3   3   3 258   3   3   3 258   3 258 258   3   3   3 258 258   3
 258   3 258 258 258 258 258   2   0   0   0   0   0   0   0   0   0   0
   0   0] 38


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=int32)

Create embedding layers for each set of tokens and sum them to create a single one.

In [48]:
nEmbeddingChannels = 256

nEmbeddingsValue = 2 ** nBits + nTokens
valueEmbedding = tf.keras.layers.Embedding(nEmbeddingsValue, nEmbeddingChannels, input_length=1)

nEmbeddingsCoord = 4
coordEmbedding = tf.keras.layers.Embedding(nEmbeddingsCoord, nEmbeddingChannels)

nEmbeddingsPos = maxSeqLen
posEmbedding = tf.keras.layers.Embedding(nEmbeddingsPos, nEmbeddingChannels)


#valueEmbed = valueEmbedding(valTokens)
#coordEmbed = coordEmbedding(coordTokens)
#posEmbed = posEmbedding(posTokens)

x = valueEmbedding + coordEmbedding + posEmbedding
x

TypeError: ignored

In [46]:
output = tf.keras.layers.Dense(maxSeqLen, tf.nn.softmax)
model = tf.keras.Model(x, output)

model

ValueError: ignored

In [ ]:
nSeq = len(valTokens)
maskDims = (nSeq, nSeq)
targetMask = t

In [ ]:
np.arange(10) % 3

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0])